# Notes
- 9010 rated all arousal as 1
- 9028 rated all arousal as 2 during cortisol runs
- 9029's placebo run's memory is missing; experimenter error - they were shown the wrong stimuli at encoding, so their memory wasn't actually tested at retrieval

exlude 9029 for memory analysis
check 9010 and 9028 for arousal related analysis

In [1]:
import scipy.io
import pandas as pd
import numpy as np

# id by trial index

In [2]:
folder="../results/dynamic_connectivity/per_subj/";            

sub_ls=["9001","9002","9003","9004","9005","9006","9007","9008","9010","9011","9013","9016","9018","9021","9022","9027","9028","9030","9031","9033","9036","9037" ,"9038","9039","9040","9041"]
print(len(sub_ls))
cond_ls=['cortisol','placebo']
stim_ls=['alc','tool']

#print(len(sub_ls))

behavior_df=pd.read_csv('../data/behavior/mem_allsub_alltrial_pill.csv')
behavior_df['RecogAcc_coded']=behavior_df['RecogAcc'].apply(lambda x:0 if x==4 else x)
pheno='RecogAcc_coded' # need to update

26


In [3]:
sub_ls=["9001","9002","9003","9004","9005","9006","9007","9008","9010","9011","9013","9016","9018","9021","9022","9027","9028","9030","9031","9033","9036","9037" ,"9038","9039","9040","9041"]
sub_ls_len=len(sub_ls)
print(sub_ls_len)
for cond in cond_ls:
    if cond=="placebo":
        pill="Placebo"
    else:
        pill="Cortisol"       
    for stim in stim_ls:
        X=np.array([])
        y=np.array([])
        id_ls=np.array([])
        for sub in sub_ls:
            #read in whole conn matrix for subj
            current_conn=scipy.io.loadmat(folder+sub+'_'+cond+'_'+stim+'_dynamic_connectome.mat')['data']
            #select behavior data
            behavior=behavior_df[(behavior_df['Subject']==int(sub))&(behavior_df['Pill']==pill)&(behavior_df['Stim']==stim)&(~behavior_df[pheno].isna())][[pheno,'Enc_Trial']]
            behavior=behavior.sort_values('Enc_Trial').reset_index(drop=True)
            conn_len=np.shape(current_conn)[0]
            
            behavior['index']=behavior['Enc_Trial'].apply(lambda x:x-1)
            # if the behavior data is longer than scan data, shorten behavior list     
            if (behavior.shape[0]>conn_len):
                behavior=behavior.loc[:conn_len-1]
                current_X=current_conn    
            # if the scan data is longer than behavior data, only select trials with behavior data
            else:
                current_X=current_conn[behavior['index'].values,:,:]
                        
            current_y=behavior[pheno].values
            current_id=behavior['Enc_Trial'].values

            #concatnate data across subj
            if X.shape[0]<1:
                X=np.array(current_X)
                y=np.array(current_y)
                id_ls=np.array(current_id)
            else:
                X=np.concatenate((X,current_X))
                y=np.concatenate((y,current_y))
                id_ls=np.concatenate((id_ls,current_id))
            if (current_X==(-np.inf)).sum()>0:
                print(sub,cond,stim)
        print(np.shape(X))
        print(np.shape(y))
        print(np.shape(id_ls))
        output_folder="../results/dynamic_connectivity/";            
        file_path = output_folder+"n"+str(sub_ls_len)+'_'+pheno+'_'+cond+"_"+stim+"_trialID_dynamic_connectome.mat"
        scipy.io.savemat(file_path, {'X': X,'y':y,'id':id_ls})

26
(1016, 377, 377)
(1016,)
(1016,)
(1034, 377, 377)
(1034,)
(1034,)
(1031, 377, 377)
(1031,)
(1031,)
(1031, 377, 377)
(1031,)
(1031,)
